In [2]:
import numpy as np
import pandas as pd
from functools import reduce

In [3]:
df_2021_1 = pd.read_excel('encuesta20211.xlsx')
df_2021_2 = pd.read_excel('encuesta20212.xlsx')
df_2022_1 = pd.read_excel('encuesta20221.xlsx')
df_2022_2 = pd.read_excel('encuesta20222.xlsx')
df_2023_1 = pd.read_excel('encuesta20231.xlsx')

## Renombrado y borrado de columnas

Empecemos arreglando algunos detalles básicos de las columnas.

In [4]:
def sym_diff_cols(*dfs):
    cols = [set(df.columns.values) for df in dfs]
    return reduce(set.union, cols) - reduce(set.intersection, cols)

sym_diff_cols(df_2021_1, df_2021_2, df_2022_1, df_2022_2, df_2023_1)

{'Actividad principal',
 'Antigüedad en la empresa actual',
 'Años en el puesto actual',
 'Años en la empresa actual',
 'Beneficios extra',
 'Con qué beneficios contas',
 'Cómo considerás que están tus ingresos laborales comparados con el semestre anterior',
 'Cómo creés que está tu sueldo con respecto al último semestre',
 'Dedicación',
 'En el último año, en tu trabajo ¿recibiste o escuchaste comentarios que considerás inapropiados, subidos de tono y/o discriminatorios?',
 'Estoy trabajando en',
 'Estoy trabajando en ',
 'Frameworks, herramientas y librerías',
 'Frameworks, herramientas y librerías que utilices en tu puesto actual',
 'IDEs',
 'Lenguajes de programación o tecnologías que utilices en tu puesto actual',
 'Lenguajes de programación o tecnologías.',
 'Me identifico',
 'Me identifico (género)',
 'Modalidad de trabajo',
 'Máximo nivel de estudios',
 'Nivel de estudios alcanzado',
 'Pagos en criptomonedas',
 'Plataformas',
 'Plataformas que utilizas en tu puesto actual',
 'P

In [27]:
mapping = {
    'region': ['Dónde estás trabajando'],
    'pais': ['Estoy trabajando en', 'Estoy trabajando en '],
    'edad': ['Tengo (edad)', 'Tengo'],
    'genero': ['Me identifico (género)', 'Me identifico'],
    'dedicacion': ['Tipo de contrato', 'Dedicación'],
    
    # s{alario}_{categoria}
    's_bruto': [
        'Salario mensual o retiro BRUTO (en tu moneda local)',
        'Último salario mensual  o retiro BRUTO (en tu moneda local)'
    ],
    's_neto': [
        'Salario mensual o retiro NETO (en tu moneda local)',
        'Último salario mensual o retiro NETO (en tu moneda local)'
    ],
    's_dolar_pagos': ['Pagos en dólares'],
    's_dolar_ultimo_valor': [
        '¿Cuál fue el último valor de dólar que tomaron?',
        'Si tu sueldo está dolarizado ¿Cuál fue el último valor del dólar que tomaron?'
    ],
    's_cripto': ['Pagos en criptomonedas'],

    # o{pinion}_{categoria}
    'o_salario': [
        '¿Qué tan conforme estás con tu sueldo?',
        '¿Qué tan conforme estás con tus ingresos laborales?'
    ],
    'o_salario_semestre_anterior': [
        'Cómo creés que está tu sueldo con respecto al último semestre',
        'Cómo considerás que están tus ingresos laborales comparados con el semestre anterior'
    ],

    # b{onos}_{categoria} 
    'b_sueldo': ['Recibís algún tipo de bono'],
    'b_sueldo_criterio': ['A qué está atado el bono'],
    'b_ajuste_inflacion': [
        '¿Tuviste ajustes por inflación durante 2021?',
        '¿Tuviste actualizaciones de tus ingresos laborales durante 2022? ',
        '¿Tuviste actualizaciones de tus ingresos laborales durante 2022?',
        '¿Tuviste ajustes por inflación durante 2020?',
        '¿Tuviste ajustes por inflación durante 2021?'
    ],
    'b_ajuste_inflacion_acum': [
        '¿De qué % fue el ajuste total?',
        '¿De qué % fue el ajuste total acumulado?'
    ],
    'b_beneficios': ['Con qué beneficios contas', 'Beneficios extra'],
    
    # p{uesto}_{categoria}
    'p_nombre': ['Trabajo de'],
    'p_años_experiencia': ['Años de experiencia'],
    'p_años_puesto': ['Tiempo en el puesto actual', 'Años en el puesto actual'],
    'p_años_empresa': ['Años en la empresa actual', 'Antigüedad en la empresa actual'],
    'p_modalidad': [
        'Modalidad de trabajo',
        '¿Cuántas veces a la semana vas a trabajar a la oficina?'
    ],
    
    # t{ecnologias}_{categoria}
    't_lenguajes': [
        'Lenguajes de programación o tecnologías.',
        'Lenguajes de programación o tecnologías que utilices en tu puesto actual'
    ],
    't_frameworks': [
        'Frameworks, herramientas y librerías',
        'Frameworks, herramientas y librerías que utilices en tu puesto actual'
    ],
    't_bases_de_datos': ['Bases de datos'],
    't_testing': ['QA / Testing'],
    
    # e{mpresa}_{categoria}
    'e_empleados': ['Cantidad de personas en tu organización'],
    'e_recomendacion': ['¿La recomendás como un buen lugar para trabajar?'],
    
    # f{ormacion}_{categoria}
    'f_max': ['Nivel de estudios alcanzado', 'Máximo nivel de estudios'],
    'f_estado': ['Estado'],
    'f_carrera': ['Carrera'],
    'f_uni': ['Universidad', 'Universidad ']
}

Pero queremos el mapeo inverso.

In [28]:
inv_mapping = {v2: k for k, v1 in mapping.items() for v2 in v1}

Ahora las que queremos borrar.

In [29]:
drop = [
    '¿Gente a cargo?', 'Plataformas', '¿En qué mes fue el último ajuste?', 
    '¿Cuántas personas a cargo tenés? ', 'Plataformas que utilizas en tu puesto actual ',
    'Plataformas que utilizas en tu puesto actual',
    'IDEs',
    'Trabajo para una empresa que no tiene oficina en mi ciudad',
    'Actividad principal',
    '¿Cuál es el compromiso que tiene tu empresa con la diversidad, la equidad y la inclusión?',
    '¿Salir o seguir contestando?',
    '¿Salir o seguir contestando?.1',
    '¿Salir o seguir contestando?.2',
    '¿Salir o seguir contestando?.3',
    '¿Contribuís a proyectos open source?',
    '¿Programás como hobbie?',
    '¿Tenés guardias?',
    'Cuánto cobrás por guardia',
    '¿Porcentaje, bruto o neto?',
    'Sufriste o presenciaste situaciones de violencia y/o acoso por motivo de',
    '¿Considerás que en tu empresa/organización hay una marcada tendencia a escuchar más a los hombres?',
    '¿Sentís que podés ser vos en tu trabajo?',
    '¿Considerás que tenés oportunidades de crecimiento siendo quien sos dentro de tu organización?',
    '¿Tenés algún tipo de discapacidad?',
    '¿Sentís que alguna vez los prejuicios culturales/sociales sobre tu orientación, género, etnia o discapacidad pudieron obstaculizar el que consigas un trabajo?',
    'En el último año, en tu trabajo ¿recibiste o escuchaste comentarios que considerás inapropiados, subidos de tono y/o discriminatorios?',
    '¿Qué tanto sentís que te está apoyando tu empresa/organización durante la pandemia?',
    '¿Instauraron algún beneficio nuevo?',
    '¿Participaste de algún Boot Camp?',
    'Realizaste cursos de especialización',
    'Si participaste de un Boot Camp, ¿qué carrera estudiaste?',
    'Si trabajás bajo un esquema híbrido ¿Cuántos días a la semana vas a la oficina?',
    'Si trabajas bajo un esquema híbrido ¿Cuántos días a la semana vas a la oficina? ',
    'Plataformas que utilizas en tu puesto actual',
    '¿Tenés guardias? ',
    '¿Cuántas personas a cargo tenés?',
    '¿En qué mes fue el último ajuste? *',
    '¿Qué porcentaje o monto recibís en criptomonedas?',
    '¿Cambió tu situación laboral a raíz de la pandemia?',
    '¿Con cuántas personas estás conviviendo?',
    '¿Con quiénes convivís?',
    '¿Cómo calificás las políticas de diversidad e inclusión?',
    '¿Cómo se vio afectada tu empresa/organización?',
    '¿Cómo venís llevando la cuarentena?',
    '¿Cómo venís llevando la pandemia?',
    '¿Qué tanto sentís que te está apoyando tu empresa/organización en esta situación?',
    '¿Qué tipo de cuarentena hiciste / estás haciendo?',
    '¿Sentís que esto te dificultó el conseguir trabajo?',
    '¿Sufriste o presenciaste situaciones de violencia laboral?',
    '¿Tenés hijos/as menores de edad?',
    '¿Tenés que compartir tu equipo de trabajo con alguien?',
    '¿Tenés un espacio dedicado para el trabajo?',
]

In [30]:
# Primero dropeamos "Tipo de contrato" de 2022.2 y 2023.1
# para que no haya name clashing; entre 2022.1 y 2022.2
# cambiaron la definición de la columna. La nueva definición
# no nos interesa tanto (planta permanente vs. terciarizado);
# sí nos interesa más si es full-time o part-time (la definición
# vieja, que renombraron a "Dedicación").
# En 2022.1 están mezcladas ambas categorías, nos quedamos con
# lo que nos interesa.
tipo_contrato = [
    'Remoto (empresa de otro país)',
    'Tercerizado (trabajo a través de consultora o agencia)',
    'Freelance', 'Participación societaria en una cooperativa'
]
mask_2021_1 = ~df_2021_1['Tipo de contrato'].isin(tipo_contrato)
mask_2021_2 = ~df_2021_2['Tipo de contrato'].isin(tipo_contrato)
mask_2022_1 = ~df_2022_1['Tipo de contrato'].isin(tipo_contrato)

dfs_no_contrato = [
    df_2021_1[mask_2021_1],
    df_2021_2[mask_2021_2],
    df_2022_1[mask_2022_1],
    df_2022_2.drop(columns=['Tipo de contrato']),
    df_2023_1.drop(columns=['Tipo de contrato'])
]

# Renombramos y dropeamos las columnas
df_cols = []
for df in dfs_no_contrato:
    renames = {}
    drops = []
    
    for col in df.columns.values:
        if col in drop:
            drops.append(col)
        
        new_name = inv_mapping.get(col)
        if new_name:
            renames[col] = new_name

    df_cols.append(df.drop(columns=drops).rename(columns=renames))
            
sym_diff_cols(*df_cols)

{'p_modalidad', 'pais', 's_cripto'}

El dataset 2022.1 sabemos que no tiene ni `pais`, ni `s_cripto` ni `s_cripto_porcentaje`; pero para `pais` sabemos que todo
el muestreo fue de Argentina. En las demás columnas agregamos NA.

Para el 2021 `p_modalidad` es tramposo, porque estuvimos en cuarentena, así que ponemos "Curentena".

Además, agregamos dos columnas más: `año` y `semestre`, indicando a qué año y a qué semestre pertenece cada observación. Esto
va a servir para cuando unamos todos los datasets.

In [33]:
df_cols[0]['p_modalidad'] = 'Cuarentena'
df_cols[1]['p_modalidad'] = 'Cuarentena'


for i in range(0, 3):
    df_cols[i]['pais'] = 'Argentina'
    df_cols[i]['s_cripto'] = np.nan

sym_diff_cols(*df_cols)

set()

In [34]:
years = [2021, 2021, 2022, 2022, 2023]
semesters = [1, 2, 1, 2, 1]
idx = range(5)

for i, y, s in zip(idx, years, semesters):
    df_cols[i]['año'] = y
    df_cols[i]['semestre'] = s

list(df_cols[0].columns.values)

['region',
 'dedicacion',
 's_bruto',
 's_neto',
 's_dolar_pagos',
 's_dolar_ultimo_valor',
 'o_salario',
 'o_salario_semestre_anterior',
 'b_sueldo',
 'b_sueldo_criterio',
 'b_ajuste_inflacion',
 'b_ajuste_inflacion_acum',
 'p_nombre',
 'p_años_experiencia',
 'p_años_empresa',
 'p_años_puesto',
 't_lenguajes',
 't_frameworks',
 't_bases_de_datos',
 't_testing',
 'e_empleados',
 'e_recomendacion',
 'b_beneficios',
 'f_max',
 'f_estado',
 'f_carrera',
 'f_uni',
 'edad',
 'genero',
 'p_modalidad',
 'pais',
 's_cripto',
 'año',
 'semestre']

## Sanitizado de datos

Primero ya podemos unir todos los datasets en uno solo.

In [35]:
df_completo = pd.concat(df_cols)
df_completo

,region,dedicacion,s_bruto,s_neto,s_dolar_pagos,s_dolar_ultimo_valor,o_salario,o_salario_semestre_anterior,b_sueldo,b_sueldo_criterio,...,f_estado,f_carrera,f_uni,edad,genero,p_modalidad,pais,s_cripto,año,semestre
0,Córdoba,Full-Time,115000,90000.0,NaN,NaN,3,2,No,No recibo bono,...,Incompleto,Licenciatura en Sistemas de Información,UTN - Universidad Tecnológica Nacional,36,Varón Cis,Cuarentena,Argentina,NaN,2021,1
2,Ciudad Autónoma de Buenos Aires,Full-Time,148000,109000.0,NaN,NaN,3,3,No,No recibo bono,...,NaN,NaN,NaN,40,Varón Cis,Cuarentena,Argentina,NaN,2021,1
3,Ciudad Autónoma de Buenos Aires,Full-Time,115620,NaN,NaN,NaN,4,3,No,No recibo bono,...,NaN,NaN,NaN,29,Varón Cis,Cuarentena,Argentina,NaN,2021,1
4,Ciudad Autónoma de Buenos Aires,Full-Time,47300,39259.0,NaN,NaN,3,3,No,No recibo bono,...,En curso,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,24,Varón Cis,Cuarentena,Argentina,NaN,2021,1
5,Ciudad Autónoma de Buenos Aires,Full-Time,100000,91713.0,NaN,NaN,1,2,Un sueldo,Performance individual,...,Incompleto,Licenciatura en Ciencias de la Computación,UBA - Universidad de Buenos Aires,29,Varón Cis,Cuarentena,Argentina,NaN,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6707,Tucumán,Full-Time,300000,249000,NaN,NaN,3,4,No,No recibo bono,...,Completo,Analista de Sistemas,Universidad Siglo 21,31,Mujer Cis,100% remoto,Argentina,NaN,2023,1
6708,Tucumán,Full-Time,2000,NaN,Cobro todo el salario en dólares,NaN,3,4,No,No recibo bono,...,En curso,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,30,Varón Cis,100% remoto,Argentina,NaN,2023,1
6709,Tucumán,Full-Time,190000,167000,Mi sueldo está dolarizado (pero cobro en moned...,250,3,2,No,No recibo bono,...,NaN,NaN,NaN,37,Mujer Cis,100% remoto,Argentina,NaN,2023,1
6710,Tucumán,Full-Time,950000,750000,Cobro parte del salario en dólares,189,3,3,Un sueldo,Aguinaldo,...,NaN,NaN,NaN,39,Prefiero no decir,100% remoto,Argentina,NaN,2023,1


### Sanitización de tipos de datos

Hay columnas que queremos forzar a números o descartarlas. En particular:

In [36]:
numeric_mask = (
    pd.to_numeric(df_completo.s_bruto, errors='coerce').isna() |
    pd.to_numeric(df_completo.s_neto, errors='coerce').isna() |
    pd.to_numeric(df_completo.edad, errors='coerce').isna() |
    pd.to_numeric(df_completo.b_ajuste_inflacion_acum, errors='coerce').isna()
)

print(df_completo[numeric_mask].shape)

(2143, 34)


No perdemos una gran cantidad de datos y sería mucho trabajo intentar arreglarlos. Usemos este nuevo dataframe filtrado.

In [37]:
df_completo_2 = df_completo[~numeric_mask]
df_completo_2.shape

(26313, 34)

Queda finalmente adaptar las columnas de modalidad de trabajo. Usamos el siguiente mapeo: si va entre 1 y 4 veces a la oficina es modalidad híbrida, 0 veces es 100% remoto y 5 veces es 100% presencial.

In [38]:
def modalidad(x):
    if not isinstance(x, (int, float)):
        return x
    if x == 0:
        return '100% remoto'
    elif 1 <= x <= 4:
        return 'Híbrido (presencial y remoto)'
    elif x == 5:
        return '100% presencial'
    return x

p_modalidad = df_completo_2['p_modalidad'].apply(modalidad)
df_completo_2.p_modalidad = p_modalidad

<ipython-input-38-5b6bd34fa3ce>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completo_2.p_modalidad = p_modalidad


Arreglemos también los factores posibles en la columna de si recibe todo/parte/nada del sueldo en criptomonedas.

In [39]:
def cripto(x):
    if pd.isnull(x):
        return 'No recibo criptomonedas'
    return x

s_cripto = df_completo_2.s_cripto.apply(cripto)
df_completo_2.s_cripto = s_cripto

<ipython-input-39-34e90979c88e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completo_2.s_cripto = s_cripto


### Guardamos el nuevo dataset

In [40]:
df_completo_2.to_excel('encuesta_sanitizada2.xlsx', index=False)